In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from pathlib import Path
from parse import parse

from alphaMusic.utils.fls_utils import load_from_pkl
import alphaMusic.evaluation as evl

In [2]:
sns.set(style='whitegrid', context='paper', font_scale=1.2,
        rc={
            'figure.figsize':(3.5,3.15), 
            'lines.linewidth':1.5,
            'font.family': 'serif',
            'text.usetex': False,
            })
pal = sns.cubehelix_palette(7, start=0.5, rot=-0.5,dark=0.3, light=.75, reverse=True, hue=1.)

In [5]:
path_to_results = Path('..','recipes','EUSIPCO22','results')
assert path_to_results.exists()

files = path_to_results.glob('N:30*RT60:0.25*DRR:1.0*.pkl')
exp_name = 'N:{n_samples}_RT60:{rt60_name}_SNR:{snr_name}_DRR:{drr_name}_nsrcs:{n_srcs}_nmics:{n_mics}_noise:{noise_type}'

results = pd.DataFrame()
for file in files:
    print(file)
    params = parse(exp_name,str(file.name))
    df = load_from_pkl(file)
    df['n_mics'] = int(params['n_mics'])
    df['n_srcs'] = int(params['n_srcs'])
    results = pd.concat([results, df], ignore_index=True)

../recipes/EUSIPCO22/results/N:30_RT60:0.25_SNR:10_DRR:1.0_nsrcs:2_nmics:6_noise:cafet_results.pkl
../recipes/EUSIPCO22/results/N:30_RT60:0.25_SNR:10_DRR:1.0_nsrcs:1_nmics:4_noise:cafet_results.pkl
../recipes/EUSIPCO22/results/N:30_RT60:0.25_SNR:10_DRR:1.0_nsrcs:4_nmics:6_noise:cafet_results.pkl
../recipes/EUSIPCO22/results/N:30_RT60:0.25_SNR:10_DRR:1.0_nsrcs:3_nmics:2_noise:cafet_results.pkl
../recipes/EUSIPCO22/results/N:30_RT60:0.25_SNR:10_DRR:1.0_nsrcs:3_nmics:6_noise:cafet_results.pkl
../recipes/EUSIPCO22/results/N:30_RT60:0.25_SNR:10_DRR:1.0_nsrcs:2_nmics:4_noise:cafet_results.pkl
../recipes/EUSIPCO22/results/N:30_RT60:0.25_SNR:10_DRR:1.0_nsrcs:1_nmics:2_noise:cafet_results.pkl
../recipes/EUSIPCO22/results/N:30_RT60:0.25_SNR:10_DRR:1.0_nsrcs:4_nmics:4_noise:cafet_results.pkl
../recipes/EUSIPCO22/results/N:30_RT60:0.25_SNR:10_DRR:1.0_nsrcs:1_nmics:6_noise:cafet_results.pkl
../recipes/EUSIPCO22/results/N:30_RT60:0.25_SNR:10_DRR:1.0_nsrcs:3_nmics:4_noise:cafet_results.pkl
../recipes

In [41]:
df = pd.DataFrame()

for r, row in results.iterrows():
    
    # compute metrics
    doas_est = row['DOAs_est']
    doas_ref = row['DOAs']
    
    metrics = evl.compute_ssl_metrics(doas_est, doas_ref)
    
    df.at[r,'algo'] = row['algo']

    df.at[r,'MAE'] = metrics['MAE']
    df.at[r,'RMSE'] = metrics['RMSE']
    df.at[r,'ACC5'] = metrics['ACC5']
    df.at[r,'ACC10'] = metrics['ACC10']
    df.at[r,'MISS'] = metrics['MISS']
    
    # params
    df.at[r,'SNR'] = row['SNR']
    df.at[r,'RT60'] = row['RT60']
    df.at[r,'n_mics'] = row['n_mics']
    df.at[r,'n_srcs'] = row['n_srcs']
    df.at[r, 'p'] = row['p']
    df.at[r, 'alpha'] = row['alpha']
    
    if row['algo'] in ['aMUSIC', 'aNormMUSIC']:
        if row['alpha'] == 1.8:
            a = 'a=1.8'
        elif row['alpha'] == 1.5:
            a = 'a=1.5'
        else:
            a = 'a=est'
        if row['p'] == 1.5:
            p = 'p=1.5'
        elif row['p'] == 1:
            p = 'p=1'
        df.at[r,'sub_algo'] = f"{row['algo']}_{a}_{p}"
    if row['algo'] in ['MUSIC', 'NormMUSIC']:
        df.at[r,'sub_algo'] = f"a{row['algo']}_2"
        

In [45]:
df

,algo,MAE,RMSE,ACC5,ACC10,MISS,SNR,RT60,n_mics,n_srcs,p,alpha,sub_algo
0,MUSIC,10.50,11.067972,0.0,50.0,0.0,10.0,0.25,6.0,2.0,NaN,NaN,aMUSIC_2
1,aMUSIC,11.00,11.045361,0.0,0.0,0.0,10.0,0.25,6.0,2.0,1.0,1.553090,aMUSIC_a=est_p=1
2,aMUSIC,11.00,11.045361,0.0,0.0,0.0,10.0,0.25,6.0,2.0,1.0,1.500000,aMUSIC_a=1.5_p=1
3,aMUSIC,11.00,11.045361,0.0,0.0,0.0,10.0,0.25,6.0,2.0,1.0,1.800000,aMUSIC_a=1.8_p=1
4,aMUSIC,14.50,14.713939,0.0,0.0,0.0,10.0,0.25,6.0,2.0,1.5,1.550441,aMUSIC_a=est_p=1.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10795,aNormMUSIC,15.00,18.641352,25.0,50.0,0.0,10.0,0.25,2.0,4.0,1.0,1.800000,aNormMUSIC_a=1.8_p=1
10796,aNormMUSIC,7.25,8.674676,25.0,50.0,0.0,10.0,0.25,2.0,4.0,1.5,1.281944,aNormMUSIC_a=est_p=1.5
10797,aNormMUSIC,10.50,13.322913,25.0,75.0,0.0,10.0,0.25,2.0,4.0,1.5,1.500000,aNormMUSIC_a=1.5_p=1.5
10798,aNormMUSIC,19.00,21.851773,0.0,50.0,0.0,10.0,0.25,2.0,4.0,1.5,1.800000,aNormMUSIC_a=1.8_p=1.5


In [43]:
res = df.groupby(['sub_algo','n_mics','n_srcs']).mean('MAE')['MAE']

In [46]:
sub_algo = df.loc[df['p'] != 1]['sub_algo'].unique()

In [47]:
table_ij_algo = pd.DataFrame()

metric = 'MAE'

c = 0
for J in [1,2,3,4]:
    for I in [2,4,6]:
        for algo in sub_algo:
            df_selection = df.loc[(df['sub_algo'] == algo) 
                                & (df['n_mics'] == I) 
                                & (df['n_srcs'] == J) ]
            err = np.mean(df_selection[metric])
            std = np.std(df_selection[metric])
            
            table_ij_algo.at[c, 'n_srcs'] = int(J)
            table_ij_algo.at[c, 'n_mics'] = int(I)
#             table_ij_algo.at[c, algo] = '%1.2f +- %1.2f' % (err, std)
            table_ij_algo.at[c, algo] = '%1.2f' % (err)
        c += 1

table_ij_algo

,n_srcs,n_mics,aMUSIC_2,aMUSIC_a=est_p=1.5,aMUSIC_a=1.5_p=1.5,aMUSIC_a=1.8_p=1.5,aNormMUSIC_2,aNormMUSIC_a=est_p=1.5,aNormMUSIC_a=1.5_p=1.5,aNormMUSIC_a=1.8_p=1.5,NaN
0,1.0,2.0,26.32,30.68,31.12,29.40,13.20,14.03,14.43,13.23,nan
1,1.0,4.0,4.83,4.68,5.63,8.15,1.52,1.22,1.17,1.17,nan
2,1.0,6.0,1.32,1.17,1.22,1.65,0.57,0.52,0.48,0.50,nan
3,2.0,2.0,30.83,34.83,32.38,33.53,30.17,24.45,29.36,27.69,nan
4,2.0,4.0,9.05,10.72,10.22,8.97,9.05,8.89,9.22,9.38,nan
5,2.0,6.0,9.35,5.90,5.74,6.12,4.42,5.36,5.38,4.88,nan
6,3.0,2.0,21.58,22.82,21.78,20.91,17.98,21.28,20.37,18.96,nan
7,3.0,4.0,16.04,13.22,14.02,14.10,6.29,5.93,5.99,5.78,nan
8,3.0,6.0,12.54,8.82,8.89,7.03,5.36,6.08,6.08,6.10,nan
9,4.0,2.0,14.53,15.36,14.41,15.16,13.05,12.64,13.20,13.98,nan


In [ ]:
sns.set(style='whitegrid', context='paper', font_scale=1.2,
        rc={
            'figure.figsize':(3.5,3.15), 
            'lines.linewidth':1.5,
            'font.family': 'sans-serif',
            'font.sans-serif': [u'Helvetica'],
            'text.usetex': False,
            })
pal = sns.cubehelix_palette(7, start=0.5, rot=-0.5,dark=0.3, light=.75, reverse=True, hue=1.)
algo_order = ['MUSIC','aMUSIC_est','aMUSIC_fix','NormMUSIC','aNormMUSIC_est','aNormMUSIC_fix','SRP_PHAT']

In [ ]:
plt.figure(figsize=(20,8))
plt.suptitle('For all mics and srcs values')

plt.subplot(131)
sns.boxplot(x='algo', y='MAE', data=df[['algo','MAE']],
           hue_order=algo_order, palette=pal)
plt.ylim([0,100])
plt.xticks(rotation=45)
plt.ylabel('Angular Error [$^\circ$] \n $\leftarrow$ the lower, the better')
plt.xlabel('Algorithim')

plt.subplot(132)
sns.barplot(x='algo', y='ACC5', data=df[['algo','ACC5']],
           hue_order=algo_order, palette=pal)
plt.ylim([0,100])
plt.xticks(rotation=45)
plt.ylabel('Accurancy < 5 $\circ$ [$\%$]')
plt.xlabel('Algorithim')

plt.subplot(133)
sns.barplot(x='algo', y='MISS', data=df[['algo','MISS']],
           hue_order=algo_order, palette=pal)
plt.ylim([0,100])
plt.xticks(rotation=45)
plt.ylabel('Miss [$\%$] \n the lower, the better')
plt.xlabel('Algorithim')

# sns.despine(offset=10, trim=False, left=True, bottom=True)
plt.tight_layout(pad=0.5)
plt.show()

In [ ]:
for n_mics in [2,4,6]:
    plt.figure(figsize=(20,5))
    plt.subplot(131)
    plt.title(f'n_mics {n_mics}')
    sns.pointplot(x='n_srcs', y='MAE', hue='algo', 
                  palette=pal,
                  data=df.loc[df['n_mics'] == n_mics],
                  markers=['^','o','X','s','d','v','*'],
                  ci=None
    )
    plt.subplot(132)
    sns.barplot(x='n_srcs', y='ACC5', hue='algo', 
                palette=pal,
                data=df.loc[df['n_mics'] == n_mics],
    )
    plt.subplot(133)
    sns.barplot(x='n_srcs', y='MISS', hue='algo', 
                palette=pal,
                data=df.loc[df['n_mics'] == n_mics],
    )
    
    plt.tight_layout()